#### Import Libraries:::

In [12]:
import numpy as np

In [13]:
#Function to read the ubyte images file:
def read_ubyte_images(file_path):
    with open(file_path, 'rb') as f:
        magic_number = int.from_bytes(f.read(4), 'big')  
        num_images = int.from_bytes(f.read(4), 'big')   
        rows = int.from_bytes(f.read(4), 'big')        
        cols = int.from_bytes(f.read(4), 'big')        
        
        # Read image data (num_images * rows * cols)
        image_data = np.frombuffer(f.read(), dtype=np.uint8)
        images = image_data.reshape(num_images, rows, cols)
    
    return images

In [14]:
#Function to read the ubyte images label file:
def read_ubyte_labels(file_path):
    with open(file_path, 'rb') as f:
        magic_number = int.from_bytes(f.read(4), 'big')  
        num_labels = int.from_bytes(f.read(4), 'big')  
        
        # Read label data (num_labels)
        label_data = np.frombuffer(f.read(), dtype=np.uint8)
    
    return label_data

In [16]:
train_images = read_ubyte_images('./data/train-images.idx3-ubyte')
train_labels = read_ubyte_labels('./data/train-labels.idx1-ubyte')
test_images = read_ubyte_images('./data/t10k-images.idx3-ubyte')
test_labels = read_ubyte_labels('./data/t10k-labels.idx1-ubyte')

In [19]:
print(train_images.shape)
print(test_images.shape)

(60000, 28, 28)
(10000, 28, 28)
